# infos

In [1]:
# Modèle basé sur les 10 espèces représentées


# Vérifier que les chemins soient correct avant toutes opérations
chemin_images = '../../images/'
chemin_csv = '../data/top10.csv'

# Définition du DataFrame echantillon (utile pour tests modèles car entrainements très rapides)
pourcentage_echantillon = 0.1 # Si 0.1 : 10% du contenu

# Dimensions des images
img_dim = (224,224)
img_shape = (224,224,3)

# Taille des batchs
batch_size=64

##### A faire sur le notebook :

- Terminer la programmation optuna
- 

# Google Colab

Si le notebook tourne sur colab, charger les fichiers images (format zip) et dezipper en suivant les cellules qui suivent :

In [ ]:
# Importer les images en format .zip
from google.colab import files
files.upload()

In [ ]:
# Monter le Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Dezipper le fichier
!unzip '/content/drive/MyDrive/SAS/images.zip' -d '/images'

In [ ]:
# Remplacer les chemins en corrélation avec les dossiers colab
chemin_images = '/images/images/'
chemin_csv = '/content/drive/MyDrive/SAS/Jul23_bds_champignons/data/top10.csv'

# Librairies à charger

In [2]:
# Librairies utilisées par les fonctions
import pandas as pd
import os
from tensorflow.keras.applications.efficientnet import preprocess_input


# Librairies utilisées pour les callbacks
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
from timeit import default_timer as timer
from tensorflow.keras.callbacks import TerminateOnNaN


# Librairies utilisées pour créer les pipelines et le modèle
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models

# Librairies utilisées pour optimise rle modèle*
import optuna
import optuna.visualization as optuna_viz
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model


# Librairies utilisées pour la création des jeux d'entrainement, de test et de validation
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter


# Librairies utilisées pour l'évaluation du modèle une fois entrainé
import matplotlib.pyplot as plt
%matplotlib inline
import random

# Fonctions

In [3]:
def import_df(chemin_images, chemin_csv, pourcentage_echantillon):
    '''Importe le fichier csv et construit 2 df :
        - Le DF basé sur le CSV original
        - Un DF echantillon comportant 10% de données aléatoires du DF original
    '''
    
    
    # import du df
    df = pd.read_csv(chemin_csv, low_memory=False)
    df['image_url'] = df['image_url'].str.replace('.../images/', chemin_images)
    print(f"Nombre d'images chargées pour df: {df.shape[0]}")
    print(f"Nb especes dans df: {df['label'].nunique()}")


    # Contruction de l'echantillon
    L = len(df)
    L_ech = int(pourcentage_echantillon * L)
    df_ech = df.sample(n=L_ech, random_state=10)
    df_ech.reset_index(inplace=True, drop=True)
    print(f"Nombre d'images chargées pour df_ech: {df_ech.shape[0]}")
    print(f"Nb especes dans df_ech: {df_ech['label'].nunique()}")




    return df, df_ech

In [52]:
def augment_img(image_path, label):

    '''Modifie les images aléatoirement dans le dataset qui sera soumis au modèle, oversample les classes sous représentées.
       image_path : chemin des images (variable définie en début de notebook),
       label : Variable contenant les classes,
   '''

    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, img_dim)         # Rappel : img_dim est définie en début de Notebook
    img = preprocess_input(img)

    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = (img - tf.math.reduce_min(img)) / (tf.math.reduce_max(img) - tf.math.reduce_min(img))


    return img, label

In [53]:
def create_tf_dataset(image_path, labels, batch_size, oversample_cls = None):
    '''
    Créé un dataset Tensorflow selon les paramètres précisés. La fonction oversample les classes sous représentées
    image_path : chemin relatif de la variable contenant les images
    labels : variable contenant les labels
    batch_size : taille des batchs
    oversample_cls : Liste contenant les classes à oversampler. Si non précisé, l'oversample sera ignoré
    '''

    image_path = image_path.tolist()  # Convertir les chemins d'images en liste
    labels = labels.tolist()          # Convertir les labels en liste


 # Oversample des classes
    if oversample_cls:
    # Compter le nombre d'exemples par classe
        class_counts = Counter(labels)

    # Calculer le nombre d'exemples à ajouter pour chaque classe à oversampler
        max_count = max(class_counts.values())
        facteurs_oversample = {cls: max_count / count for cls, count in class_counts.items() if cls in oversample_cls}

    # Répéter les exemples des classes à oversampler pour atteindre le nombre maximum
        oversampled_image_paths = []
        oversampled_labels = []
        for img_path, label in zip(image_path, labels):
            facteurs_oversample = facteurs_oversample.get(label, 1.0)
            nb_copies = int(facteurs_oversample)
            for _ in range(nb_copies):
                oversampled_image_paths.append(img_path)
                oversampled_labels.append(label)

        image_path = oversampled_image_paths
        labels = oversampled_labels



    # Construction du Dataset    
    dataset = tf.data.Dataset.from_tensor_slices((image_path, labels))
    dataset = dataset.map(augment_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=len(image_path))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

In [4]:
def controle_presence_fichiers(df, chemin_images):

    ''' Controle que les fichiers images soient bien présents sur le disque.'''

    image_directory = chemin_images
    missing_files = []

# Parcourir chaque ligne du DataFrame
    for index, row in df.iterrows():
        image_path = os.path.join(image_directory, row['image_lien'])
    
        if not os.path.exists(image_path):
            missing_files.append(image_path)

    # Afficher les fichiers non trouvés
    if missing_files:
        print("\nFichiers non trouvés :")
        for file_path in missing_files:
            print(file_path)
    else:
        print("\nTous les fichiers sont présents.")

# Optuna - Fonctions

La bibliothèque Optuna est une bibliothèque open source en Python qui est principalement utilisée pour l'optimisation des hyperparamètres, également connue sous le nom d'optimisation automatique des hyperparamètres (AutoML). Elle permet d'automatiser le processus de recherche des meilleures combinaisons d'hyperparamètres pour les modèles d'apprentissage automatique, ce qui peut grandement améliorer les performances des modèles.

**Voici les principales utilisations et fonctionnalités de la bibliothèque Optuna :**
- Optimisation des hyperparamètres : Optuna peut rechercher automatiquement les meilleures valeurs d'hyperparamètres pour un modèle donné en minimisant ou maximisant une fonction objectif. Les hyperparamètres sont des paramètres qui ne sont pas appris par le modèle lui-même, mais qui affectent ses performances, tels que le taux d'apprentissage, la profondeur du réseau de neurones, la taille du lot, etc.

- Gestion des essais : Optuna gère la recherche des hyperparamètres en effectuant une recherche efficace dans l'espace des hyperparamètres en utilisant des algorithmes d'optimisation tels que l'optimisation des arbres de décision, l'optimisation bayésienne, etc. Il maintient un historique des essais antérieurs pour guider la recherche.

- Intégration avec les frameworks de machine learning : Optuna peut être utilisé avec différents frameworks d'apprentissage automatique, tels que TensorFlow, PyTorch, Scikit-Learn, XGBoost, LightGBM, etc. Il est donc polyvalent et peut être utilisé pour optimiser divers types de modèles.

- Extensible : Optuna est extensible, ce qui signifie que vous pouvez définir votre propre espace d'hyperparamètres à rechercher et définir des objectifs personnalisés en fonction de votre problème spécifique.

- Parallélisme : Optuna prend en charge le parallélisme, ce qui signifie que vous pouvez effectuer plusieurs essais en parallèle pour accélérer le processus d'optimisation.

- Visualisation des résultats : Optuna offre des outils de visualisation pour vous permettre d'analyser les résultats de l'optimisation, tels que les graphiques d'importance des hyperparamètres, les courbes d'apprentissage, etc.


En résumé, Optuna est une bibliothèque puissante pour l'optimisation des hyperparamètres qui permet d'automatiser et de rationaliser le processus de recherche des meilleures configurations de modèle, ce qui peut vous faire gagner du temps et améliorer considérablement les performances de vos modèles d'apprentissage automatique.

### Modèle pré-entrainé

In [ ]:
efficientNetv2 = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/classification/2"
pre_trained_model = hub.KerasLayer(efficientNetv2, input_shape=(224, 224, 3), trainable=False)

### Fonction objectif

Vous devez définir une fonction objectif que vous souhaitez optimiser. Cette fonction prendra les hyperparamètres comme arguments et renverra une valeur que vous souhaitez minimiser ou maximiser.

In [ ]:
def create_model(trial):
    # Paramètres à optimiser
    
     # Nombre de couches cachées
    n_hidden = trial.suggest_int('n_hidden', 1, 5)
    n_units = trial.suggest_int('n_units', 32, 128)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    dropout_rates = [trial.suggest_float(f'dropout_layer_{i}', 0.0, 0.5) for i in range(num_hidden_layers)]


    # Créer le modèle EfficientNetv2 pré-entraîné avec des couches gelées (doit être importé aupravant)
    base_model = pre_trained_model


    # Ajouter des couches personnalisées pour la classification
    x = base_model.output
    
    for i in range(n_hidden):
        # Ajouter une couche dense avec le nombre d'unités spécifié
        x = tf.keras.layers.Dense(n_units, activation='relu')(x)
                # Ajouter une couche de dropout avec le taux spécifié
        x = tf.keras.layers.Dropout(dropout_rates[i])(x)


    # Sortie du modèle
    predictions = Dense(10, activation='softmax')(x)
    model = Model(inputs=pre_trained_model.input, outputs=predictions)


    # Compilation du modèle
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])


    return model

In [ ]:
def objective(trial):
    model = create_model(trial)
    optuna_history = model.fit(ds_train, 
                               validation_data = ds_val,
                               batch_size=batch_size,
                               epochs=20,
                               callbacks = [tensorboard, early_stopping, reduceLR, checkpoint, time_callback, TON]
                               verbose=0)
    
    score = model.evaluate(ds_test)[1]
    return score

### Etude

Vous devez créer un objet d'étude Optuna et spécifier la direction de l'optimisation (minimisation ou maximisation) 

La différence entre les modes "minimize" et "maximize" dans une étude Optuna réside dans la manière dont Optuna interprète la fonction objectif que vous cherchez à optimiser.

**Minimize (Minimiser) :**

Lorsque vous spécifiez direction='minimize' lors de la création de votre étude Optuna, vous indiquez à Optuna que vous cherchez à minimiser la valeur de la fonction objectif.
Cela signifie que vous cherchez à obtenir la plus petite valeur possible de la fonction objectif. Par exemple, si vous utilisez la perte d'un modèle de machine learning comme fonction objectif, vous souhaitez minimiser cette perte (c'est-à-dire obtenir une perte aussi faible que possible).


**Maximize (Maximiser) :**

En revanche, lorsque vous spécifiez direction='maximize', vous indiquez à Optuna que vous cherchez à maximiser la valeur de la fonction objectif.
Cela signifie que vous cherchez à obtenir la plus grande valeur possible de la fonction objectif. Par exemple, si vous cherchez à maximiser la précision d'un modèle de classification, vous souhaitez obtenir une précision aussi élevée que possible.
Le choix entre "minimize" et "maximize" dépend du problème que vous résolvez et de la manière dont vous définissez votre fonction objectif. Par exemple, si vous cherchez à minimiser les erreurs, les pertes ou les coûts, vous utiliserez généralement "minimize". Si vous cherchez à maximiser les performances, les scores ou les gains, vous utiliserez généralement "maximize".




**Voici un exemple concret :** supposons que vous entraîniez un modèle de classification et que votre fonction objectif est la précision du modèle. Dans ce cas, vous voudriez spécifier direction='maximize' car vous cherchez à obtenir la meilleure précision possible. D'un autre côté, si vous optimisez la perte du modèle, vous spécifierez direction='minimize' car vous voulez minimiser la perte.

In [ ]:
study = optuna.create_study()

# Callbacks

### Tensorboard

In [55]:
%load_ext tensorboard
log_dir = '../tensor_board_logs'
tensorboard = callbacks.TensorBoard(log_dir = log_dir)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### EarlyStopping

In [56]:
early_stopping = EarlyStopping(monitor = 'val_accuracy', 
                               min_delta = 0.03,
                               patience = 8,
                               verbose = 1,
                               mode = 'auto',
                               restore_best_weights = True)

### Reduce LearningRate

In [57]:
reduceLR = ReduceLROnPlateau(monitor = 'val_loss',
                             min_delta = 0.01,
                             patience = 5,
                             factor = 0.15, 
                             cooldown = 3,
                             verbose = 1)

### Checkpoint

In [58]:
checkpoint = ModelCheckpoint(filepath='../model/checkpoint_model', monitor='val_accuracy', save_best_only=True, verbose=1)

### Timer

In [79]:
class TimingCallback(Callback):
    def __init__(self):
        super().__init__()
        self.logs = []

    def on_epoch_begin(self, epoch, logs=None):
        self.starttime = timer()

    def on_epoch_end(self, epoch, logs=None):
        endtime = timer()
        elapsed_time = endtime - self.starttime
        self.logs.append(elapsed_time)
        print(f"Epoch {epoch + 1} took {elapsed_time:.2f} seconds")

time_callback = TimingCallback()

### Terminate on NaN

In [60]:
TON = TerminateOnNaN()

# Pipeline Dataset

### Import des DataFrames

In [5]:
# Rappel : Utiliser df_ech pour les tests (entrainement rapide du modèle mais accuracy faible)
df, df_ech = import_df(chemin_images, chemin_csv, pourcentage_echantillon)

Nombre d'images chargées pour df: 64372
Nb especes dans df: 10
Nombre d'images chargées pour df_ech: 6437
Nb especes dans df_ech: 10


C:\Users\guill\AppData\Local\Temp\ipykernel_1236\465607992.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['image_url'] = df['image_url'].str.replace('.../images/', chemin_images)


In [6]:
# Préciser sur quelles données travailler (df_ech est un echantillon permettant de réduire le temps d'entrainement pour effectuer des tests)
# Commenter/Decommenter la ligne souhaitée
#donnees_training = df_ech
donnees_training = df

In [7]:
# Représentation des classes
print('Facteurs d\'oversampling des classes : \n',
      'Agaricales : x1 \n',
      'Agaricus: x2.5 \n',
      'Amanita : x2 \n',
      'Cortinarius: x1.5 \n',
      'Entoloma : x2.5 \n',
      'Inocybe: x2 \n',
      'Mycena : x2 \n',
      'Popyporales: x1.8 \n',
      'Psathyrella : x2 \n',
      'Russula: x1.5 \n')

donnees_training.groupby('label').count()

Facteurs d'oversampling des classes : 
 Agaricales : x1 
 Agaricus: x2.5 
 Amanita : x2 
 Cortinarius: x1.5 
 Entoloma : x2.5 
 Inocybe: x2 
 Mycena : x2 
 Popyporales: x1.8 
 Psathyrella : x2 
 Russula: x1.5 



,image_lien,image_url
label,,
Agaricales,11517,11517
Agaricus,4692,4692
Amanita,4987,4987
Cortinarius,7352,7352
Entoloma,4209,4209
Inocybe,5607,5607
Mycena,5342,5342
Polyporales,6864,6864
Psathyrella,5564,5564


In [ ]:
# Controle de la présence des fichiers images
controle_presence_fichiers(donnees_training, chemin_images)

# On supprime ensuite la colonne image_lien qui ne sert qu'à controler la présence des fichiers.
donnees_training.drop('image_lien', axis=1, inplace=True)

In [71]:
# Définir les classes à oversampler :
oversample_cls = ['Agaricus', 'Amanita', 'Cortinarius', 'Entoloma', 'Inocybe', 'Mycena', 'Polyporales', 'Psathyrella', 'Russula']

### Construction des jeux de données (train, test et validation)

In [72]:
data = donnees_training.drop('label', axis=1)
target = donnees_training['label']

s = LabelEncoder()
target = s.fit_transform(target) # Encodage de la variable 'label'

# On construit le jeu d'entrainnement. X_temp et y_temps servent pour la construction des jeux de test et validation
X_train, X_temp, y_train, y_temp = train_test_split(data, target, test_size=0.25, random_state=10)

# On split les temp en 50% pour test, 50% pour validation
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=10)

### Construction des dataset Tensorflow

In [73]:
batch_size = 64
# Les datasets sont créés à partir de la fonction create_tf_dataset définie dans la partie 'Fonctions'
ds_train= create_tf_dataset(X_train.image_url, y_train, batch_size)
ds_test = create_tf_dataset(X_test.image_url, y_test, batch_size)
ds_val = create_tf_dataset(X_val.image_url, y_val, batch_size)

# Optuna - Otimisation

Utilisez la méthode optimize de l'objet d'étude en spécifiant la fonction objectif et le nombre d'essais que vous souhaitez effectuer

In [ ]:
# /!\ Attention l'entrainement peut être très très long
study.optimize(objective, n_trials=15, n_jobs=-1)

### Résultats

Une fois l'optimisation terminée, vous pouvez accéder aux meilleurs hyperparamètres et à la meilleure valeur obtenue

In [ ]:
best_params = study.best_params
best_value = study.best_value
print(best_params)

In [ ]:
optuna_viz.plot_param_importances(study)

# Optuna - Construction best_model

In [ ]:
# Construire avec les meilleurs paramètres

In [ ]:
# Compiler le modèle
best_model = create_model(study.best_trial)


In [ ]:
best_model.summary()

# Evaluation du modèle

In [82]:
test_loss, test_accuracy = best_model.evaluate(ds_test)
print("Test accuracy:", test_accuracy)

13/13 [==============================] - 34s 3s/step - loss: 1.7338 - accuracy: 0.4099
Test accuracy: 0.40993788838386536


In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Sauvegarde du modèle

In [30]:
# Changer le nom du modèle si il s'agit d'un nouvel entrainement

# Save en dur
#nom_modele = '../model/gpot_v01_optuna_echantillon'

# Save sur GDrive
nom_modele =  '/content/drive/MyDrive/SAS/model/gpot_v01_optuna'

In [ ]:
model.save(nom_modele)